References:
*   https://dzone.com/articles/pyspark-dataframe-tutorial-introduction-to-datafra
*   https://stackoverflow.com/questions/44156365/when-to-cache-a-dataframe
*   https://changhsinlee.com/pyspark-dataframe-basics/
*   https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/


*   https://stackoverflow.com/questions/44156365/when-to-cache-a-dataframe
*   https://sparkbyexamples.com/spark/spark-case-when-otherwise-example/
*   https://stackoverflow.com/questions/37707305/pyspark-multiple-conditions-in-when-clause
*   List item







## Tahap Persiapan

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import pandas as pd

In [0]:
custData = {'CustomerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William', 5: 'Owen'},
         'Sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male', 5: 'male'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1}}

In [0]:
custInfo = {'CustomerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Age': {0: 22, 1: 38, 2: 26, 3: 35, 4: 35},
         'Fare': {0: 7.3, 1: 71.3, 2: 7.9, 3: 53.1, 4: 8.0},
         'Pclass': {0: 3, 1: 1, 2: 3, 3: 1, 4: 3},
         'RegisteredDate': {0: '2017-04-01', 1: '2017-04-15', 2: '2017-05-12', 3: '2017-02-09', 4: '2017-04-05',}}


In [0]:
pd_custData = pd.DataFrame(custData, columns=custData.keys())
pd_custInfo = pd.DataFrame(custInfo, columns=custInfo.keys())


In [63]:
pd_custData

,CustomerId,Name,Sex,Survived
0,1,Owen,male,0
1,2,Florence,female,1
2,3,Laina,female,1
3,4,Lily,female,1
4,5,William,male,0
5,6,Owen,male,1


In [31]:
pd_custInfo

,CustomerId,Age,Fare,Pclass,RegisteredDate
0,1,22,7.3,3,2017-04-01
1,2,38,71.3,1,2017-04-15
2,3,26,7.9,3,2017-05-12
3,4,35,53.1,1,2017-02-09
4,5,35,8.0,3,2017-04-05


In [0]:
df_custData = spark.createDataFrame(pd_custData)
df_custInfo = spark.createDataFrame(pd_custInfo)

In [65]:
df_custData.show()

+----------+--------+------+--------+
|CustomerId|    Name|   Sex|Survived|
+----------+--------+------+--------+
|         1|    Owen|  male|       0|
|         2|Florence|female|       1|
|         3|   Laina|female|       1|
|         4|    Lily|female|       1|
|         5| William|  male|       0|
|         6|    Owen|  male|       1|
+----------+--------+------+--------+



In [66]:
print ('Number of records: %d ' % df_custData.count()) 

Number of records: 6 


In [67]:
print ('Number of column: %d ' % len(df_custData.columns))

Number of column: 4 


In [68]:
df_custData.printSchema()

root
 |-- CustomerId: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Survived: long (nullable = true)



In [69]:
df_custInfo.printSchema()

root
 |-- CustomerId: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Pclass: long (nullable = true)
 |-- RegisteredDate: string (nullable = true)



In [70]:
df_custData.describe().show()

+-------+------------------+--------+------+------------------+
|summary|        CustomerId|    Name|   Sex|          Survived|
+-------+------------------+--------+------+------------------+
|  count|                 6|       6|     6|                 6|
|   mean|               3.5|    null|  null|0.6666666666666666|
| stddev|1.8708286933869707|    null|  null|0.5163977794943223|
|    min|                 1|Florence|female|                 0|
|    max|                 6| William|  male|                 1|
+-------+------------------+--------+------+------------------+



## Selecting Columns 

In [71]:
df_custData.select('CustomerId', 'Name').show()

+----------+--------+
|CustomerId|    Name|
+----------+--------+
|         1|    Owen|
|         2|Florence|
|         3|   Laina|
|         4|    Lily|
|         5| William|
|         6|    Owen|
+----------+--------+



## Filtering Data 

In [72]:
df_custData.select('CustomerId', 'Name') \
           .filter("Sex = 'female'") \
           .show()

+----------+--------+
|CustomerId|    Name|
+----------+--------+
|         2|Florence|
|         3|   Laina|
|         4|    Lily|
+----------+--------+



In [73]:
df_custData.select('CustomerId', 'Name', 'Sex', 'Survived') \
           .filter("Sex == 'female' AND Name == 'Lily'") \
           .show()

+----------+----+------+--------+
|CustomerId|Name|   Sex|Survived|
+----------+----+------+--------+
|         4|Lily|female|       1|
+----------+----+------+--------+



In [74]:
df_custData.select('CustomerId', 'Name', 'Sex', 'Survived') \
           .filter("CustomerId >2") \
           .show()

+----------+-------+------+--------+
|CustomerId|   Name|   Sex|Survived|
+----------+-------+------+--------+
|         3|  Laina|female|       1|
|         4|   Lily|female|       1|
|         5|William|  male|       0|
|         6|   Owen|  male|       1|
+----------+-------+------+--------+



In [75]:
df_custData.select('CustomerId', 'Name', 'Sex', 'Survived') \
           .filter("CustomerId BETWEEN 2 and 4") \
           .show()

+----------+--------+------+--------+
|CustomerId|    Name|   Sex|Survived|
+----------+--------+------+--------+
|         2|Florence|female|       1|
|         3|   Laina|female|       1|
|         4|    Lily|female|       1|
+----------+--------+------+--------+



In [76]:
df_custInfo.filter("RegisteredDate BETWEEN '2017-04-01' AND '2017-04-30'") \
           .orderBy("RegisteredDate") \
           .show()

+----------+---+----+------+--------------+
|CustomerId|Age|Fare|Pclass|RegisteredDate|
+----------+---+----+------+--------------+
|         1| 22| 7.3|     3|    2017-04-01|
|         5| 35| 8.0|     3|    2017-04-05|
|         2| 38|71.3|     1|    2017-04-15|
+----------+---+----+------+--------------+



## Grouping the Data

In [77]:
df_custInfo.groupBy("Pclass") \
           .agg({'*' : 'count', 'Age' : 'avg', 'Fare' : 'sum', 'CustomerId' : 'min'} ) \
           .show()

+------+--------+---------------+------------------+---------+
|Pclass|count(1)|min(CustomerId)|          avg(Age)|sum(Fare)|
+------+--------+---------------+------------------+---------+
|     1|       2|              2|              36.5|    124.4|
|     3|       3|              1|27.666666666666668|     23.2|
+------+--------+---------------+------------------+---------+



In [78]:
df_custInfo.groupBy("Pclass") \
           .agg({'*' : 'count', 'Age' : 'avg', 'Fare' : 'sum', 'CustomerId' : 'min'} ) \
           .toDF('PClass', 'Num Row', 'Min-CustID', 'Avg_Age', 'Total Fare') \
           .show()

+------+-------+----------+------------------+----------+
|PClass|Num Row|Min-CustID|           Avg_Age|Total Fare|
+------+-------+----------+------------------+----------+
|     1|      2|         2|              36.5|     124.4|
|     3|      3|         1|27.666666666666668|      23.2|
+------+-------+----------+------------------+----------+



In [80]:
df_custData.show()

+----------+--------+------+--------+
|CustomerId|    Name|   Sex|Survived|
+----------+--------+------+--------+
|         1|    Owen|  male|       0|
|         2|Florence|female|       1|
|         3|   Laina|female|       1|
|         4|    Lily|female|       1|
|         5| William|  male|       0|
|         6|    Owen|  male|       1|
+----------+--------+------+--------+



In [82]:
df_custData.groupBy('Sex', 'Survived') \
           .count() \
           .orderBy('Sex') \
           .show()


+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|       1|    3|
|  male|       0|    2|
|  male|       1|    1|
+------+--------+-----+



## Joining Data

In [85]:
df_custData.join(df_custInfo, df_custData.CustomerId == df_custInfo.CustomerId) \
           .orderBy(df_custData.CustomerId) \
           .show()

+----------+--------+------+--------+----------+---+----+------+--------------+
|CustomerId|    Name|   Sex|Survived|CustomerId|Age|Fare|Pclass|RegisteredDate|
+----------+--------+------+--------+----------+---+----+------+--------------+
|         1|    Owen|  male|       0|         1| 22| 7.3|     3|    2017-04-01|
|         2|Florence|female|       1|         2| 38|71.3|     1|    2017-04-15|
|         3|   Laina|female|       1|         3| 26| 7.9|     3|    2017-05-12|
|         4|    Lily|female|       1|         4| 35|53.1|     1|    2017-02-09|
|         5| William|  male|       0|         5| 35| 8.0|     3|    2017-04-05|
+----------+--------+------+--------+----------+---+----+------+--------------+



In [87]:
df_custData.join(df_custInfo, df_custData.CustomerId == df_custInfo.CustomerId, 'left_outer') \
           .orderBy(df_custData.CustomerId) \
           .show()

+----------+--------+------+--------+----------+----+----+------+--------------+
|CustomerId|    Name|   Sex|Survived|CustomerId| Age|Fare|Pclass|RegisteredDate|
+----------+--------+------+--------+----------+----+----+------+--------------+
|         1|    Owen|  male|       0|         1|  22| 7.3|     3|    2017-04-01|
|         2|Florence|female|       1|         2|  38|71.3|     1|    2017-04-15|
|         3|   Laina|female|       1|         3|  26| 7.9|     3|    2017-05-12|
|         4|    Lily|female|       1|         4|  35|53.1|     1|    2017-02-09|
|         5| William|  male|       0|         5|  35| 8.0|     3|    2017-04-05|
|         6|    Owen|  male|       1|      null|null|null|  null|          null|
+----------+--------+------+--------+----------+----+----+------+--------------+



## Nonequi joins

In [88]:
df_custData.join(df_custInfo, df_custData.CustomerId <= df_custInfo.CustomerId) \
           .orderBy(df_custData.CustomerId) \
           .show()

+----------+--------+------+--------+----------+---+----+------+--------------+
|CustomerId|    Name|   Sex|Survived|CustomerId|Age|Fare|Pclass|RegisteredDate|
+----------+--------+------+--------+----------+---+----+------+--------------+
|         1|    Owen|  male|       0|         4| 35|53.1|     1|    2017-02-09|
|         1|    Owen|  male|       0|         3| 26| 7.9|     3|    2017-05-12|
|         1|    Owen|  male|       0|         2| 38|71.3|     1|    2017-04-15|
|         1|    Owen|  male|       0|         1| 22| 7.3|     3|    2017-04-01|
|         1|    Owen|  male|       0|         5| 35| 8.0|     3|    2017-04-05|
|         2|Florence|female|       1|         2| 38|71.3|     1|    2017-04-15|
|         2|Florence|female|       1|         3| 26| 7.9|     3|    2017-05-12|
|         2|Florence|female|       1|         4| 35|53.1|     1|    2017-02-09|
|         2|Florence|female|       1|         5| 35| 8.0|     3|    2017-04-05|
|         3|   Laina|female|       1|   

## Union

In [89]:
df_custData.union(df_custData).show()

+----------+--------+------+--------+
|CustomerId|    Name|   Sex|Survived|
+----------+--------+------+--------+
|         1|    Owen|  male|       0|
|         2|Florence|female|       1|
|         3|   Laina|female|       1|
|         4|    Lily|female|       1|
|         5| William|  male|       0|
|         6|    Owen|  male|       1|
|         1|    Owen|  male|       0|
|         2|Florence|female|       1|
|         3|   Laina|female|       1|
|         4|    Lily|female|       1|
|         5| William|  male|       0|
|         6|    Owen|  male|       1|
+----------+--------+------+--------+

